In [1]:
IS_COLAB = False
try:
    import google.colab
    IS_COLAB = True
except:
    pass
if IS_COLAB:
    !pip install tensorflow-gpu
    !pip install jsonpickle

In [2]:
if IS_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
if IS_COLAB:
    %cd /content/drive/My\ Drive/MastersThesis
    !ls
    !pwd
    import sys
    sys.path.append('./')

/content/drive/My Drive/MastersThesis
 Autoencoderstart.ipynb   for_training.ipynb    README.md
 callbacks.py		  global_functions.py   training_data.py
 config.py		  model_combined.png    training.py
 constants.py		  models.py	        vizualizer.py
 datasets.py		  output_dir	       'zaujmave citanie.pdf'
 Dokumentovanie		  predictors.py
 experiments.py		  __pycache__
/content/drive/My Drive/MastersThesis


In [7]:
import tensorflow as tf
if tf.test.is_gpu_available():
    print("GPU OK")
else:
    print("GPU NOT FOUND")

GPU NOT FOUND


In [8]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [0]:
import global_functions
global_functions.on_start()

Using TensorFlow backend.


In [6]:
import training
training.TrainingFashionMnist.classifier_layers_off()

Nove OK
start_batch_index: 42000 ,end_batch_index: 48000
Autoencoder trainable weights:  0
Train on 6000 samples, validate on 10000 samples
Epoch 1/100
6000/6000 - 34s - loss: 1.1226 - accuracy: 0.6237 - val_loss: 2.5688 - val_accuracy: 0.1067
Epoch 2/100
6000/6000 - 32s - loss: 0.6661 - accuracy: 0.7650 - val_loss: 3.1758 - val_accuracy: 0.1000
Epoch 3/100
6000/6000 - 32s - loss: 0.5379 - accuracy: 0.8133 - val_loss: 4.1915 - val_accuracy: 0.1000
Epoch 4/100
Restoring model weights from the end of the best epoch: 1. Last epoch: 4. val_accuracy best value: 0.10670000314712524.


NameError: ignored

In [0]:
import vizualizer, experiments, constants, datasets

results = experiments.ExperimentBase.load_experiment_results(constants.ExperimentsPaths.Mnist.CLASSIFIER, False)
vizualizer.Vizualizer.vizualize([result.train_history for result in results], 0.5)


In [0]:
def make_batches(to_batch, batch_size = 128):
    return [to_batch[i * batch_size:(i + 1) * batch_size] for i in range((len(to_batch) + batch_size - 1) // batch_size )]

batch_size = 128
to_batch = tf.data.Dataset.from_tensor_slices((train_images, keras.utils.to_categorical(train_labels, 10) ))
train_batched = to_batch.batch(batch_size)

In [0]:
epochs = 10
for epoch in range(epochs):
    print("Epoch: {}", epoch)
    for step, (images, labels) in enumerate(train_batched):
        set_autoencoder_trainable(True)
        with tf.GradientTape() as tape:
          prediction = autoencoder(images)
          loss_autoencoder = autoencoder.loss(images, prediction)
          gradients = tape.gradient(loss_autoencoder, autoencoder.trainable_variables)
          autoencoder.optimizer.apply_gradients(zip(gradients, autoencoder.trainable_variables))  
        set_autoencoder_trainable(False)
        with tf.GradientTape() as tape:
          predict_class = classifier(images)
          loss_classifier = classifier.loss(labels, predict_class)  
#           classiefier_trainable = classifier_head.trainable_variables
          classiefier_trainable = classifier.trainable_variables
          gradients = tape.gradient(loss_classifier, classiefier_trainable)
          classifier.optimizer.apply_gradients(zip(gradients  ,classiefier_trainable) ) 
          if step % 10 == 0:
              print("Step: {}, loss autoencoder: {}, loss classifier: {}".format(step, loss_autoencoder.numpy().mean(), loss_classifier.numpy().mean()))


In [0]:
for epoch in range(5):
  print("Classifier")
  historyClassifier = classifier.fit(x = train_images, y = train_labels_one_hot_encoding, batch_size=128, epochs=1, validation_data= (test_images, test_labels_one_hot_encoding))

In [0]:
predictions_auto_encoder, predictions_classifier = auto_classifier.predict(test_images)
predictions_classifier = np.argmax(np.round(predictions_classifier),axis=1)
correct = np.where(predictions_classifier==test_labels)[0]
print( "Found {} correct labels".format(len(correct)))

In [0]:

test_eval = classifier.evaluate(test_images, test_labels_one_hot_encoding, verbose=0)
print('Test loss:', test_eval[0])
print('Test accuracy:', test_eval[1])
predicted_classes = classifier.predict(test_images)
predicted_classes = np.argmax(np.round(predicted_classes),axis=1)
predicted_classes = predicted_classes.astype('uint8')
correct = [predicted_classes[i] == value for i, value in enumerate(test_labels)]
print("Correct: {}".format( correct.count(True)))




In [0]:
predictions = autoencoder.predict(test_images)
# rand_start = np.random.randint(0, len(predictions) - 10)
# for i in range(rand_start, rand_start + 10):
#   plotImage(predictions[i].reshape(28, 28), "Predictions")
#   plotImage(test_images[i].reshape(28, 28), "Original")


num_images = 5
#np.random.seed(42)
random_test_images = np.random.randint(test_images.shape[0], size=num_images)

decoded_imgs = predictions


plt.figure(figsize=(28, 16))

for i, image_idx in enumerate(random_test_images):
    # plot original image
    ax = plt.subplot(2, num_images, i + 1)
    plt.title("Original")
    plt.imshow(process_for_plotting(test_images[image_idx]))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    
    # # plot encoded image
    # ax = plt.subplot(3, num_images, num_images + i + 1)
    # plt.imshow(encoded_imgs[image_idx].reshape(8, 4))
    # plt.gray()
    # ax.get_xaxis().set_visible(False)
    # ax.get_yaxis().set_visible(False)

    # plot reconstructed image
    ax = plt.subplot(2, num_images, num_images + i + 1)
    plt.imshow(process_for_plotting(decoded_imgs[image_idx])) #reshape(28, 28)
    plt.title("Predicted")
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [0]:

from experiments import *
import jsonpickle
exp = ExperimentAutoencoder(DatasetProviderClass(FashionMnistDataset), basic_model_provider, generate_data_for_trainig)
exp.train(BasicTrainParameters(1, 128, True, 'mylog.json', True) )



In [0]:
from experiments import *
exp = ExperimentAutoencoder(DatasetProviderClass(FashionMnistDataset), basic_model_provider, generate_data_for_trainig)
# exp.train(BasicTrainParameters(1, 128, True, 'mylog.json', True) )
# exp.train(BasicTrainParameters(2, 128, True, 'mylog.json', True) )
train_result = exp.train(BasicTrainParameters(3, 128, True, 'mylog.json', True) )


In [0]:
exp_local = train_result[1]
exp_local

In [0]:
import experiments
import datasets
import importlib
importlib.reload(experiments)
for rate in [1]:
    exp = experiments.ExperimentAutoencoder(experiments.DatasetProviderClass(datasets.FashionMnistDataset), experiments.BasicModelProvider(), experiments.generate_data_for_trainig)
    exp.train(experiments.BasicTrainParametersOneModel(100, 128, 10, True, rate, 0.001, 'mylog.json', True))
    #exp.train(experiments.BasicTrainParametersOneModel(100, 128, True, rate, 'mylog.json', True))

In [0]:
import vizualizer
import experiments
import importlib
importlib.reload(vizualizer)
results = experiments.ExperimentBase.load_experiment_results('classifier.json')
vizualizer.Vizualizer.vizualize([result.train_history for result in results], 0.5)